<div style="font-family: 'Segoe UI', system-ui, sans-serif; padding: 28px 32px; background: linear-gradient(135deg, #0078D4 0%, #106EBE 100%); border-radius: 12px; color: white; margin-bottom: 8px;">
  <div style="font-size: 13px; font-weight: 600; letter-spacing: 1.5px; text-transform: uppercase; opacity: 0.85; margin-bottom: 8px;">Microsoft Agent Framework</div>
  <div style="font-size: 30px; font-weight: 700; line-height: 1.2; margin-bottom: 6px;">🔌 에이전트에서 MCP 활용하기</div>
  <div style="font-size: 15px; opacity: 0.9; margin-top: 10px;">Model Context Protocol(MCP)을 사용하여 에이전트를 외부 도구와 연결하고,<br /> 또한, 에이전트 자체를 MCP 서버로 노출하는 방법을 학습합니다.</div>
</div>


<div style="font-family: 'Segoe UI', system-ui, sans-serif; margin-top: 4px;">

## 📖 MCP(Model Context Protocol) 개요

<div style="background: #EFF6FC; border-left: 4px solid #0078D4; border-radius: 0 8px 8px 0; padding: 16px 20px; margin: 12px 0; color: #323130;">
  <div style="font-weight: 600; color: #0078D4; margin-bottom: 6px;">💡 MCP란?</div>
  <p style="margin: 0; line-height: 1.7;">
    모델 컨텍스트 프로토콜(MCP)은 애플리케이션이 대규모 언어 모델(LLM)에 <strong>도구(Tool)</strong>와 <strong>컨텍스트 데이터</strong>를 제공하는 방식을 정의하는 <strong>개방형 표준</strong>입니다. 이를 통해 외부 도구를 에이전트에 일관되고 확장 가능한 방식으로 통합할 수 있습니다.
  </p>
  <p style="margin: 10px 0 0; line-height: 1.7;">
    <strong>Microsoft Agent Framework</strong>는 MCP 서버와의 통합을 지원하여, 에이전트가 외부 도구 및 서비스에 손쉽게 액세스할 수 있도록 합니다.
  </p>
</div>

<div style="background: #FFF4CE; border-left: 4px solid #F2A900; border-radius: 0 8px 8px 0; padding: 14px 20px; margin: 12px 0; color: #323130;">
  <div style="font-weight: 600; color: #835B00; margin-bottom: 4px;">⚠️ 주의사항</div>
  <p style="margin: 0; line-height: 1.7;">외부 MCP 서비스에 연결할 경우에는 <strong>비용, 보안, 데이터 프라이버시</strong> 등의 고려사항이 있을 수 있습니다. 외부 서비스 연결 전 반드시 해당 정책을 확인하고 신중하게 활용하세요.</p>
</div>

</div>


## MCP 사용하기

<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

### 🔌 MCP 도구 유형

에이전트 프레임워크는 다음 **3가지 유형**의 MCP 연결 방식을 지원합니다.

<div style="display: flex; gap: 12px; margin: 16px 0; flex-wrap: wrap;">
  <div style="flex: 1; min-width: 180px; background: #F3F2F1; border-radius: 8px; padding: 14px 18px; border-top: 3px solid #0078D4;">
    <div style="font-weight: 700; color: #0078D4; font-size: 14px;">🖥️ MCPStdioTool</div>
    <div style="color: #605E5C; font-size: 13px; margin-top: 4px;">로컬 프로세스 (표준 입출력)</div>
  </div>
  <div style="flex: 1; min-width: 180px; background: #F3F2F1; border-radius: 8px; padding: 14px 18px; border-top: 3px solid #0078D4;">
    <div style="font-weight: 700; color: #0078D4; font-size: 14px;">🌐 MCPStreamableHTTPTool</div>
    <div style="color: #605E5C; font-size: 13px; margin-top: 4px;">HTTP / SSE 기반 원격 서버</div>
  </div>
  <div style="flex: 1; min-width: 180px; background: #F3F2F1; border-radius: 8px; padding: 14px 18px; border-top: 3px solid #0078D4;">
    <div style="font-weight: 700; color: #0078D4; font-size: 14px;">🔄 MCPWebsocketTool</div>
    <div style="color: #605E5C; font-size: 13px; margin-top: 4px;">WebSocket 기반 실시간 연결</div>
  </div>
</div>

---

### 🖥️ MCPStdioTool — 로컬 MCP 서버

`MCPStdioTool`을 사용하면 **표준 입출력(Stdio)** 을 통해 로컬 프로세스로 실행되는 MCP 서버에 연결할 수 있습니다.

아래 예시에서는 Agent Framework(Python)와 함께 사용할 수 있는 로컬 MCP 서버인 `uvx mcp-server-calculator`를 사용하여 **간단한 수학 계산**을 수행합니다.

</div>


In [ ]:
# stdio로 로컬 MCP 서버를 사용하는 예제, 주요 코드 일부

async with (
    # 로컬 uvx mcp-server-calculator라는 MCP 서버를 MCPStdioTool로 연결합니다. 
    # 이 서버는 간단한 수학 계산을 수행하는 MCP 도구입니다.
    MCPStdioTool(
        name="calculator",
        command="uvx",
        args=["mcp-server-calculator"]
    ) as mcp_server,
    
    # 에이전트를 생성하며, mcp_server를 도구로 추가합니다. 
    Agent(
        client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        name="MathAgent",
        instructions="당신은 유용한 수학 도우미입니다. 계산을 해결할 수 있습니다.",
        tools=mcp_server
    ) as agent,
):
    result = await agent.run("15 * 23 + 45은 무엇입니까?")
    print(result)

# 현재 폴더에는 완성된 local_mcp_example.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\local_mcp_example.py

<div style="font-family: 'Segoe UI', system-ui, sans-serif; margin: 4px 0;">

<div style="background: #F3F2F1; border-radius: 8px; padding: 16px 20px;">
  <div style="font-weight: 600; color: #323130; font-size: 14px; margin-bottom: 10px;">✅ 실행 결과</div>

```powershell
(.venv) PS C:\GitHub\MAF_Workshop\MAF> python 1.py
15 × 23 + 45의 값은 390입니다.
```

</div>
</div>


<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

---

### 🌐 MCPStreamableHTTPTool — HTTP/SSE MCP 서버

`MCPStreamableHTTPTool`을 사용하면 **SSE(Server-Sent Events)** 를 통해 HTTP 기반의 원격 MCP 서버에 연결할 수 있습니다.

이번 예시에서는 공개적으로 제공되는 **Microsoft Learn MCP 서버**를 사용하여 Microsoft 기술 문서를 검색하는 방법을 보여줍니다.

<div style="background: #EFF6FC; border-left: 4px solid #0078D4; border-radius: 0 8px 8px 0; padding: 14px 20px; margin: 14px 0; color: #323130;">
  <div style="font-weight: 600; color: #0078D4; margin-bottom: 4px;">ℹ️ Microsoft Learn MCP</div>
  <p style="margin: 0; line-height: 1.7;">Microsoft Learn MCP는 별도의 인증이나 API 키 없이 사용할 수 있는 <strong>공개 MCP 서버</strong>입니다. 누구나 자유롭게 Microsoft 공식 기술 문서를 검색하는 에이전트를 구성할 수 있습니다.</p>
</div>

</div>


In [ ]:
# HTTP 기반의 MCP 서버를 사용하는 예제입니다. 주요 코드 일부...

async with (
    AzureCliCredential() as credential,
    MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
    ) as mcp_server,
    Agent(
        client=AzureAIAgentClient(credential=credential),
        name="DocsAgent",
        instructions="당신은 Microsoft Learn에 관한 질문에 도움을 주는 에이전트입니다.",
    ) as agent,
):
    result = await agent.run("Azure 스토리지 계정을 az cli를 사용하여 생성하는 방법은 무엇입니까?")
    print(result)

# 현재 폴더에는 완성된 http_mcp_example.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\http_mcp_example.py

<div style="font-family: 'Segoe UI', system-ui, sans-serif; margin: 4px 0;">

<div style="background: #F3F2F1; border-radius: 8px; padding: 16px 20px;">
  <div style="font-weight: 600; color: #323130; font-size: 14px; margin-bottom: 10px;">✅ 실행 결과</div>

```bash
Azure 스토리지 계정을 **Azure CLI**를 사용하여 생성하려면 아래와 같은 단계를 따르면 됩니다.
---
### 1. Azure CLI 로그인
    az login
브라우저가 열리고, 계정으로 로그인이 진행됩니다.
...... (생략)
```

</div>
</div>


<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

---

### 🔄 MCPWebsocketTool — WebSocket MCP 서버

`MCPWebsocketTool`을 사용하면 **WebSocket** 연결을 통해 실시간으로 MCP 서버와 통신할 수 있습니다.

<div style="background: #F3F2F1; border-radius: 8px; padding: 16px 20px; margin: 14px 0; color: #323130; line-height: 1.8;">
  <div style="font-weight: 600; margin-bottom: 8px; color: #323130;">📌 이 섹션의 학습 계획</div>
  <ul style="margin: 0; padding-left: 20px; color: #605E5C;">
    <li>WebSocket 기반 MCP 서버는 외부에서 쉽게 찾기 어렵기 때문에, <strong>직접 구현</strong>하는 것이 테스트에 더 용이합니다.</li>
    <li>이어지는 다음 과정에서 <strong>기존 에이전트를 MCP 서버로 전환</strong>하는 실습을 진행하며, 해당 에이전트를 WebSocket 기반 MCP 서버로 전환해 볼 것입니다.</li>
    <li>지금은 코드 예시를 먼저 살펴보고, WebSocket MCP 서버가 준비된 후 함께 실행해 봅니다.</li>
  </ul>
</div>

</div>


In [ ]:
"""WebSocket 기반의 MCP 서버를 사용하는 예제입니다."""

async def websocket_mcp_example():

    async with (
        MCPWebsocketTool(
            name="restaurant-server",
            url="ws://localhost:8765",  # 실제 WebSocket MCP 서버 URL로 변경하세요
            load_prompts=False,         # 서버가 prompts/list를 지원하지 않으므로 비활성화
        ) as mcp_server,
        Agent(
            client=AzureOpenAIChatClient(credential=AzureCliCredential()),
            name="RestaurantAgent",
            instructions="당신은 레스토랑에서 서빙을 제공하는 에이전트입니다.",
            tools=mcp_server
        ) as agent,
    ):
        result = await agent.run("오늘의 스페셜 메뉴는 무엇입니까?")
        print(result)

# 현재 폴더에는 완성된 websocket_mcp_example.py 파일이 제공됩니다.
# 실행을 위해서는 새로운 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
# 그러나, 지금 실행해봐야 WebSocket MCP 서버가 아직 구현되지 않았기 때문에 연결에 실패할 것입니다.
# 해당 서버를 먼저 구현한 후에 그때 실행해 보도록 하죠.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\websocket_mcp_example.py

## 🚀 MCP 제공하기


<div style="padding: 22px 28px; background: linear-gradient(135deg, #F3F2F1 0%, #EDEBE9 100%); border-radius: 10px; border-left: 5px solid #0078D4;">
  <p style="color: #323130; line-height: 1.7; margin: 0;">
    지금까지는 에이전트가 외부 MCP 서버를 <em>소비</em>하는 방법을 살펴봤습니다. <br />
    이번에는 반대로, <strong>기존 에이전트를 MCP 서버로 노출</strong>하여 MCP 도구를 지원하는 다른 시스템에서 에이전트를 활용하는 방법을 학습합니다.
  </p>
</div>

---

### 📋 기존 에이전트를 MCP 서버로 노출

`as_mcp_server()` 메서드를 사용하면 에이전트를 MCP 서버로 변환할 수 있습니다.<br />
이를 통해 MCP 호환 클라이언트에서 해당 에이전트를 **도구(Tool)** 로 호출할 수 있게 됩니다.

<div style="background: #EFF6FC; border-left: 4px solid #0078D4; border-radius: 0 8px 8px 0; padding: 14px 20px; margin: 14px 0; color: #323130;">
  <div style="font-weight: 600; color: #0078D4; margin-bottom: 4px;">ℹ️ 진행 순서</div>
  <ol style="margin: 6px 0 0; padding-left: 20px; line-height: 1.8;">
    <li>MCP 서버로 노출할 <strong>에이전트를 생성</strong>합니다.</li>
    <li>필요한 경우 <strong>Function 도구</strong>를 추가합니다.</li>
    <li><code>as_mcp_server()</code>를 호출하여 <strong>MCP 서버로 변환</strong>합니다.</li>
    <li>MCP 서버로 <strong>실행</strong>하고 요청을 기다립니다.</li>
  </ol>
</div>

먼저 MCP 서버로 노출할 에이전트와 도구를 생성합니다.  
예제는 `오늘의 스페셜 메뉴정보`와 `메뉴 가격`을 고정적으로 반환하는 간단한 도구를 탑재한 에이전트의 예입니다.




In [ ]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 오늘의 스페셜 메뉴를 반환하는 도구
def get_specials() -> Annotated[str, "메뉴에서 스페셜 메뉴를 반환합니다. 🍽️✨"]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

# 메뉴 아이템의 가격을 반환하는 도구. 9.99 달러로 고정
def get_item_price(
    menu_item: Annotated[str, "메뉴 아이템의 이름입니다. 🍴"],
) -> Annotated[str, "메뉴 아이템의 가격을 반환합니다. 💰"]:
    return "$9.99"

# RestaurantAgent 에이전트 생성 및 도구 제공
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="RestaurantAgent",
    instructions="🍴당신은 메뉴에 대한 질문에 답변하는 레스토랑 에이전트입니다.",
    tools=[get_specials, get_item_price],
)

<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

### 🔄 에이전트를 MCP 서버로 전환

`as_mcp_server()`를 호출하면 에이전트를 MCP 서버 인스턴스로 변환할 수 있습니다.  
에이전트에 설정된 **이름(name)** 과 **설명(description)** 은 MCP 서버의 메타데이터로 활용됩니다.

</div>


In [ ]:
# as_mcp_server() 메서드를 호출하면 에이전트를 MCP 서버로 노출할 수 있습니다.
server = agent.as_mcp_server()

<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

### ▶️ MCP 서버 실행

아래 코드를 통해 MCP 서버를 **WebSocket 서버**로 시작합니다.  
서버가 실행된 후에는 `websocket_mcp_example.py`와 같은 MCP 호환 클라이언트에서 이 에이전트를 **도구(Tool)** 로 호출할 수 있습니다.

</div>


In [ ]:
async def mcp_asgi(scope, receive, send):
    if scope["type"] == "websocket":
        async with websocket_server(scope, receive, send) as (read_stream, write_stream):
            await server.run(read_stream, write_stream, server.create_initialization_options())

if __name__ == "__main__":
    print("🍴 Restaurant MCP 서버 시작: ws://localhost:8765")
    uvicorn.run(mcp_asgi, host="127.0.0.1", port=8765)

# 현재 폴더에는 완성된 restaurant_mcp_server.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\restaurant_mcp_server.py

<div style="font-family: 'Segoe UI', system-ui, sans-serif;">

---

## 🧪 MCP 서버 테스트

서버와 클라이언트를 각각 실행하여 전체 흐름을 검증해 봅니다.

---

### 1️⃣ MCP 서버 실행 확인

방금 전 단계에서 `restaurant_mcp_server.py`를 실행했다면 터미널에 서버 시작 메시지가 출력됩니다:

<div style="background: #F3F2F1; border-radius: 8px; padding: 14px 18px; margin: 10px 0;">
  <img src="images/mcp-001.png" alt="MCP 서버 시작 화면" style="max-width: 100%; border-radius: 6px;"/>
</div>

---

### 2️⃣ MCP 클라이언트 연결 테스트

서버가 실행 중인 상태에서, **새로운 터미널**을 열고 MCP 클라이언트(`websocket_mcp_example.py`)를 실행하여 MCP 서버를 도구로 호출하는지 테스트합니다.

<div style="background: #F3F2F1; border-radius: 8px; padding: 14px 20px; margin: 10px 0;">
  <div style="font-weight: 600; color: #323130; margin-bottom: 8px; font-size: 14px;">💻 새 터미널에서 실행</div>

```bash
.\.venv\Scripts\Activate.ps1
python .\MAF\websocket_mcp_example.py
```

</div>

<div style="background: #EFF6FC; border-left: 4px solid #0078D4; border-radius: 0 8px 8px 0; padding: 12px 18px; margin: 14px 0;">
  <div style="font-weight: 600; color: #0078D4; margin-bottom: 4px;">⚠️ 주의</div>
  <p style="margin: 0; color: #323130; line-height: 1.7;">기존 터미널에는 MCP 서버가 실행 중입니다. 클라이언트 실행을 위해 반드시 <strong>새로운 터미널 창</strong>을 열어야 합니다.</p>
</div>

실행 후 터미널에 MCP 서버를 도구로 호출한 결과가 출력됩니다:

<div style="background: #F3F2F1; border-radius: 8px; padding: 14px 18px; margin: 10px 0;">
  <img src="images/mcp-002.png" alt="MCP 클라이언트 실행 결과" style="max-width: 100%; border-radius: 6px;"/>
</div>

</div>
